In [18]:
import os
import re
import pandas as pd
import eclabfiles as ecf

def _get_default_data_param(columns):
    col_names = {}
    for col in columns:
        if match := re.match(r'Ewe[^|]*', col):
            col_names['voltage'] = match.group()
        elif match := re.match(r'I[^|]*', col):
            col_names['current'] = match.group()
        elif match := re.match(r'Re\(Z(we-ce)?\)[^|]*', col):
            col_names['real'] = match.group()
        elif match := re.match(r'-Im\(Z(we-ce)?\)[^|]*', col):
            col_names['imag'] = match.group()
        elif match := re.match(r'Phase\(Z(we-ce)?\)[^|]*', col):
            col_names['phase'] = match.group()
        elif match := re.match(r'\|Z(we-ce)?\|[^|]*', col):
            col_names['abs'] = match.group()
        elif match := re.match(r'time[^|]*', col):
            col_names['time'] = match.group()
        elif match := re.match(r'(z )?cycle( number)?[^|]*', col):
            col_names['cycle'] = match.group()
        elif match := re.match(r'freq[^|]*', col):
            col_names['frequency'] = match.group()
        elif match := re.match(r'Ns', col):
            col_names['Ns'] = match.group()
    return col_names

def load(path=None, files=None, cont_time=True):
    if path is None:
        path = self.eis_params["path"]

    if isinstance(files, list):
        end_time = 0
        data = []
        for f in files:
            d = load(path + f)
            if cont_time:
                start_time = d[0].time.iloc[0]
                for c in d:
                    c.time += end_time - start_time
                end_time = d[-1].time.iloc[-1]

            data += d

        return data

    if files is not None:
        path = path + files

    ext = os.path.splitext(path)[1][1:]

    if ext in {"csv", 'txt'}:
        data = pd.read_csv(path, sep=',', encoding='unicode_escape')
    elif ext in {'mpr', 'mpt'}:
        data = ecf.to_df(path)
    else:
        raise ValueError(f"Datatype {ext} not supported")

    if data.empty:
        raise ValueError(f"File {path} has no data")

    col_names = _get_default_data_param(data.columns)
    if col_names.get("cycle") is None:
        raise ValueError(f"No cycles detetected in file {path}.")

    data.set_index([col_names["cycle"], col_names.get("Ns")], inplace=True)
    data.sort_index(inplace=True)
    return data



In [19]:
from eisplottingtool import EISFrame

path1 = r"G:\Collaborators\Sauter Ulrich\Projects\EIS Tail\Data"
path2 = r"C:\Users\ueli\Desktop\Sauter Ulrich\Projects\EIS Tail\Data"
file = r"\20201204_Rabeb_LLZTO_Batch4_rAcetonitryle-3days_Li300C_3mm_0p7th_PT_C15.mpr"

print(file)

data = load(path2 + file)


\20201204_Rabeb_LLZTO_Batch4_rAcetonitryle-3days_Li300C_3mm_0p7th_PT_C15.mpr


In [ ]:
print(data.columns)

In [ ]:
print(data.loc[1])

In [ ]:
print(data.loc[0, 1])

In [ ]:
print(data.loc[(0, 1), :])

In [ ]:
print(data.loc[(0, 1), col_names["frequency"]])

In [ ]:
print(data.loc[([1, 2, 3], 1), col_names["frequency"]])

In [ ]:
print(data.loc[(slice(None), 1), col_names["frequency"]])